In [1]:
! python --version

Python 3.11.5


# Environment Setup

## Upgrade pip

In [1]:
! pip install --upgrade pip

# Install latest boto3,awscli, boto3-core

In [2]:
! pip install --no-build-isolation --force-reinstall \
    "boto3>=1.33.6" \
    "awscli>=1.31.6" \
    "botocore>=1.33.6"

  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.8.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 740.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 953.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 386.4 kB/s eta 0:00:0000:0100:01
Using cached PyYAML-6.0.1-cp311-cp311-macosx_11_0_arm64.whl (167 kB)
Using cached s3transfer-0.8.2-py3-none-a

# Load the Library

In [3]:
import json
import os
import sys

import boto3
import botocore

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

# Anthropic is the AI safety and research company behind Claude - the frontier model used by millions of businesses and consumers for its emphasis on safety and performance.

In [79]:
claude_instant_prompt = """

Human: Please provide a summary of the following text.
<text>

Guardrails for Amazon Bedrock offer a robust framework for implementing safeguards in generative AI applications, aligning with responsible AI policies and use cases. These guardrails facilitate controlled user-Foundation Model (FM) interactions by filtering out undesirable content and will soon include redacting personally identifiable information (PII), enhancing privacy. Multiple guardrails, each configured for specific use cases, can be created, allowing continuous monitoring for policy violations. The safeguard features within Guardrails encompass Denied Topics, enabling the definition of undesirable topics; Content Filters, with configurable thresholds for filtering harmful content in categories like hate, insults, sexual, and violence; and upcoming features like Word Filters and PII Redaction. The latter will allow blocking specific words and redacting PII in FM-generated responses, contributing to content safety. Guardrails are compatible with various large language models on Amazon Bedrock, including Titan, Anthropic Claude, Meta Llama 2, AI21 Jurassic, Cohere Command FMs, as well as fine-tuned FMs and Agents.AWS extends intellectual property indemnity, specifically uncapped for copyright claims, covering generative outputs from services like Amazon Titan models and CodeWhisperer Professional. This indemnity protects customers from third-party copyright claims linked to outputs generated in response to customer-provided inputs, emphasizing responsible usage and avoiding inputting infringing data or disabling filtering features. The standard IP indemnity safeguards customers from third-party claims regarding IP infringement, encompassing copyright claims for the services and their training data.
</text>

Assistant:"""



In [81]:
body = json.dumps({
    "prompt": claude_instant_prompt,
    "max_tokens_to_sample":256,
    "top_k":250,
    "stop_sequences":[], #define phrases that signal the model to conclude text generation.
    "temperature":0, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
    "top_p":0.9 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
})

In [119]:
response = bedrock_runtime.invoke_model(
    body=body,
	modelId="anthropic.claude-v2", # You can  also use "anthropic.claude-instant-v1",
    # anthropic.claude-instant-v1 is faster and cheaper yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document question-answering.
    accept="application/json", 
    contentType="application/json"
)


### Parse the Stream Data and conver to List - Uncomment to Run it.

In [1]:
# claude_stream = response.get('body')
# claude_stream_data = list(claude_stream)
# claude_stream_data

# -------- Output ##########
#[b'{"completion":" Here is a summary of the key points from the text:\\n\\n- Amazon Bedrock\'s guardrails provide a framework for implementing safeguards in generative AI applications, aligning with responsible AI policies and use cases. \\n\\n- The guardrails allow controlled user-Foundation Model interactions by filtering out undesirable content. \\n\\n- Soon the guardrails will also redact personally identifiable information to enhance privacy.\\n\\n- Multiple guardrails can be created, each configured for specific use cases, allowing continuous monitoring for policy violations.","stop_reason":"stop_sequence","stop":"\\n\\nHuman:"}']

## Description of Parse script
### The llm_output_parser function captures and parses the output, ensuring proper line wrapping.
### It redirects the standard output temporarily to capture the printed content,
### then restores the original standard output. The parsed output is printed with line wrapping.
### Exception handling is included to handle potential errors during capturing and wrapping.

In [117]:
# claude_stream = response.get('body')
# claude_stream_data = list(claude_stream)
# claude_stream_data
# # parse_output=list(claude_stream_data)
# # parse_output

from io import StringIO
import sys
import textwrap


def llm_output_parser(*args, width: int = 100, **kwargs):
    """
    llm_output_parser function:
    Parses and prints output with line wrapping to a specified width.

    Parameters:
    - *args: Variable-length argument list for print function.
    - width (int): Width for line wrapping (default 100).
    - **kwargs: Keyword arguments for print function.

    Returns:
    None

    Example Usage:
    llm_output_parser("This is a sample output for llm_output_parser function.", width=50)
    """
    buffer = StringIO()

    try:
        # Redirect sys.stdout to capture the output
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    except Exception as e:
        # Handle any exceptions that may occur during capturing
        print(f"Error capturing output: {e}")
        return
    finally:
        # Restore the original sys.stdout
        sys.stdout = _stdout

    try:
        # Wrap lines and print the parsed output
        for line in output.splitlines():
            print("\n".join(textwrap.wrap(line, width=width)))
    except Exception as e:
        # Handle any exceptions that may occur during line wrapping
        print(f"Error wrapping lines: {e}")

In [118]:
response_body = json.loads(response.get('body').read())

llm_output_parser(response_body.get('completion'))

 Here is a summary of the key points from the text:

- Amazon Bedrock's guardrails provide a framework for implementing safeguards in generative AI
applications, aligning with responsible AI policies and use cases.

- The guardrails allow controlled user-Foundation Model interactions by filtering out undesirable
content.

- Soon the guardrails will also redact personally identifiable information to enhance privacy.

- Multiple guardrails can be created, each configured for specific use cases, allowing continuous
monitoring for policy violations.
